In [ ]:
import os
import json
import time
import config
from datetime import datetime
from log_utils import (
    read_log_messages, normalize_template, normalize_template_v1, normalize_template_v2, save_templates, extract_event_templates
)
from ollama_utils import start_ollama_server, stop_ollama_server
from autogen import AssistantAgent, ConversableAgent, register_function
from evaluation import evaluate_and_save
from drain_utils import parse_logs
from autogen.coding import LocalCommandLineCodeExecutor
from pathlib import Path
from codecarbon import EmissionsTracker

In [ ]:
# --- Configuration ---
llm_config = config.LLM_CONFIG
task = config.TASK_PROMPT
LOG_DIR = config.LOG_DIR
RESULT_DIR = config.RESULT_DIR
os.makedirs(RESULT_DIR, exist_ok=True)

DESIGN = "TA-CG"
model = llm_config["config_list"][0]["model"].replace(":", "-")
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
project_name = DESIGN.capitalize()
exp_name = f"{project_name}_{model}_{timestamp}"

input_log_file = "HDFS_200_sampled_CG.log"
log_path = os.path.join(LOG_DIR, input_log_file)

In [ ]:
# --- Agent Creation ---
def create_code_executor_agent():
    executor = LocalCommandLineCodeExecutor(
        timeout=10,
        work_dir=Path(config.WORK_DIR)
    )
    return ConversableAgent(
        name="code_executor",
        description="I execute the code provided.",
        code_execution_config={"executor": executor},
        human_input_mode="NEVER",
        llm_config=False,
    )

def create_tool_caller_agent(llm_config):
    return AssistantAgent(
        name="tool_caller",
        system_message="You are a helpful assistant. You can call tools to parse logs. Return 'TERMINATE' when the task is done.",
        llm_config=llm_config,
        human_input_mode="NEVER",
    )

In [ ]:
# --- Log Reading ---
logs = read_log_messages(log_path)

In [ ]:
# Start Ollama server
proc = start_ollama_server()
time.sleep(5)  # Give it some time to initialize

In [ ]:
# --- With CodeCarbon ---
tracker = EmissionsTracker(project_name=exp_name, output_dir=RESULT_DIR, save_to_file=True)
tracker.start()
try:
    # Create tool caller agent
    tool_caller = create_tool_caller_agent(llm_config)
    code_executor = create_code_executor_agent()

    register_function(
        parse_logs,
        caller=tool_caller,
        executor=code_executor,
        description="Parse the log file with Drain algorithm.",
    )

    arguments = {
        "input_dir": "/home/user/Desktop/AgentGreen/logs",
        "log_file": "HDFS_200_sampled_CG.log",
        "log_format": r"<Date> <Time> <Pid> <Level> <Component>: <Content>",
        "regex": [r"blk_-?\d+", r"(\d+\.){3}\d+(:\d+)?"],
        "output_dir": "/home/user/Desktop/AgentGreen/results/Drain_result/",
        "depth": 4,
        "similarity_threshold": 0.5
    }

    response = code_executor.initiate_chat(
        recipient=code_executor,
        message={
            "function_call": {
                "name": "parse_logs",
                "arguments": json.dumps(arguments) 
            }
        },
        max_turns=1,
        summary_method="last_msg"
    )
    print(response.summary)
    
finally:
    emissions = tracker.stop()
    print(f"\n🌱 Emissions: {emissions} kg CO2")
    

In [ ]:
# Stop server
stop_ollama_server(proc)

In [ ]:
# --- Evaluation ---
drain_structured_csv = os.path.join(RESULT_DIR, "Drain_result", f"{input_log_file}_structured.csv")
parsed_templates = extract_event_templates(drain_structured_csv)
save_templates(parsed_templates, llm_config, DESIGN, RESULT_DIR)

In [ ]:
ground_truth_file = os.path.join(LOG_DIR, "HDFS_200_sampled.log_structured.csv")
results = evaluate_and_save(normalize_template, parsed_templates, ground_truth_file, DESIGN, RESULT_DIR)
results_v1 = evaluate_and_save(normalize_template_v1, parsed_templates, ground_truth_file, DESIGN, RESULT_DIR)
results_v2 = evaluate_and_save(normalize_template_v2, parsed_templates, ground_truth_file, DESIGN, RESULT_DIR)